In [56]:
import os
import sys
import torch
import pickle
import math
import unicodedata
import pandas as pd
import numpy as np

from args import get_args, get_ent_args
from torchtext import data

from entity_detection.simple_qa_ner import SimpleQADataset
from relation_prediction.simple_qa_relation import SimpleQaRelationDataset
from fuzzywuzzy import fuzz
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.corpus import stopwords
from relation_prediction.model import RelationClassifier
from entity_detection.model import EntityDetection
from utils import tokenize_text, www2fb, get_index, strip_accents, find_ngrams, calc_tf_idf, pick_best_name, ins, get_span, get_names

In [57]:
fb_path = "indexes/fb_graph.pkl"
index_entpath = "indexes/entity_2M.pkl"
index_reachpath = "indexes/reachability_2M.pkl"
index_namespath = "indexes/names_2M.pkl"
fb_path = "indexes/fb_graph.pkl"
index_ent = get_index(index_entpath)
index_names = get_index(index_namespath)
index_reach = get_index(index_reachpath)
fb_graph = get_index(fb_path)

loading index from: indexes/entity_2M.pkl
loading index from: indexes/names_2M.pkl
loading index from: indexes/reachability_2M.pkl
loading index from: indexes/fb_graph.pkl


In [64]:
stopwords = set(stopwords.words('english'))

In [65]:
def answer(relation, tokens, question):
    pred_relation = relation
    query_tokens = tokens

    N = min(len(query_tokens), 3)

    C = []  # candidate entities
    for n in range(N, 0, -1):
        ngrams_set = find_ngrams(query_tokens, n)
        for ngram_tuple in ngrams_set:
            ngram = " ".join(ngram_tuple)
            ngram = strip_accents(ngram)
            print(ngram)
            # unigram stopwords have too many candidates so just skip over
            if ngram in stopwords:
                continue
            ## PROBLEM! - ngram doesnt exist in index - at test-2592 - KeyError: 'p.a.r.c.e. parce'
            try:
                cand_mids = index_ent[ngram]  # search entities
            except:
                continue
            C.extend(cand_mids)
        if (len(C) > 0):
            break
        break

    C_pruned = []
    for mid in set(C):
        if mid in index_reach.keys():  # PROBLEM: don't know why this may not exist??
            count_mid = C.count(mid)  # count number of times mid appeared in C
            C_pruned.append((mid, count_mid))
            if pred_relation in index_reach[mid]:
                count_mid = C.count(mid)  # count number of times mid appeared in C
                C_pruned.append((mid, count_mid))

    print(C_pruned)
    num_entities_fbsubset = 1959820  # 2M - 1959820 , 5M - 1972702
    C_tfidf_pruned = []
    
    for mid, count_mid in C_pruned:
        if mid in index_names.keys():
            cand_ent_name = pick_best_name(question, index_names[mid])
            tfidf = calc_tf_idf(query_tokens, cand_ent_name, count_mid, num_entities_fbsubset, index_ent)
            C_tfidf_pruned.append((mid, cand_ent_name, tfidf))

    C_tfidf_pruned.sort(key=lambda t: -t[2])
    print(C_tfidf_pruned)

    pred_ent, name_ent, score = C_tfidf_pruned[0]

    print(pred_ent)
    print(name_ent)
    
    key = (pred_ent, pred_relation)
    print(key)
    if key not in fb_graph:
        return "UNKNOWN"
    result_mid = fb_graph[key]
    result_mid = list(result_mid)

    result = get_names(fb_graph, result_mid)[0]
    return result

In [66]:
#a = answer("fb:people.person.place_of_birth", ["sasha vujacic"], "where was sasha vujacic born")
a = answer("fb:music.release.region", ["dead combo"], "What is a region that dead combo was released in?")

print(a)

dead combo
[('fb:m.031j8nn', 1), ('fb:m.031j8nn', 1), ('fb:m.0gql46', 1)]
[('fb:m.031j8nn', 'dead combo', 0.0), ('fb:m.031j8nn', 'dead combo', 0.0), ('fb:m.0gql46', 'dead combo', 0.0)]
fb:m.031j8nn
dead combo
('fb:m.031j8nn', 'fb:music.release.region')
United Kingdom
